### Importando bibliotecas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install plotly --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [ ]:
import pandas as pd
from collections import Counter
import time

### Dados

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
colunas_desejadas = ['ID_ALUNO', 'IN_FINANCIAMENTO_ESTUDANTIL','IN_DEFICIENCIA', 'NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO']

In [ ]:
# Carregando dataset do Drive (O Colab não suporta upload de datasets muito grandes, por isso deve vir direto do Drive)
microdados = pd.read_parquet('/content/drive/MyDrive/PIBIC/dataCenso2018_colunasEspecificas3.parquet')

In [ ]:
# Removendo Duplicatas
microdados = microdados.drop_duplicates(subset=['ID_ALUNO'])

### Ataque de Inferência de Atributo - Baseado em Anexos do Produto 02 do TED 8750 - PRICE

#### Código baseado nos conceitos matemáticos (TED)

In [ ]:
def suc_prior_deterministic(data, sensitive_attr):
    """
    Sucesso determinístico a priori do adversário em um ataque ACU.
    """
    sensitive_values = data[sensitive_attr].unique()
    return 1 if len(sensitive_values) == 1 else 0

def suc_post_deterministic(data, qid_attrs, sensitive_attr):
    """
    Sucesso determinístico a posteriori do adversário em um ataque ACU.
    """
    groups = data.groupby(qid_attrs, dropna=False)  # Agrupa pelos QID e inclui NaN como grupo distinto
    total_records = len(data)  # Número total de registros
    success_count = 0  # Contador de sucessos

    for _, group in groups:  # Percorre cada grupo
        if len(group[sensitive_attr].unique()) == 1:  # Se o grupo tem apenas 1 valor sensível
            success_count += len(group)  # Conta todos os registros do grupo como sucesso

    return success_count / total_records  # Retorna a proporção de acertos

def suc_prior_probabilistic(data, sensitive_attr):
    """
    Sucesso probabilístico a priori do adversário em um ataque ACU.
    """
    value_counts = data[sensitive_attr].value_counts()
    total_records = len(data)
    return value_counts.max() / total_records if total_records > 0 else 0

def suc_post_probabilistic(data, qid_attrs, sensitive_attr):
    """
    Sucesso probabilístico a posteriori do adversário em um ataque ACU.
    """
    groups = data.groupby(qid_attrs, dropna=False)  # Inclui NaN como grupo distinto
    total_records = len(data)
    total_success = 0

    for _, group in groups:
        group_size = len(group)
        if group_size > 0:
            value_counts = group[sensitive_attr].value_counts(dropna=False)  # Inclui NaN ao contar
            total_success += value_counts.max()

    return total_success / total_records if total_records > 0 else 0

def degradation_privacy_deterministic(data, qid_attrs, sensitive_attr):
    """
    Degradação determinística de privacidade em um ataque ACU.
    """
    prior = suc_prior_deterministic(data, sensitive_attr)
    post = suc_post_deterministic(data, qid_attrs, sensitive_attr)
    return post - prior

def degradation_privacy_probabilistic(data, qid_attrs, sensitive_attr):
    """
    Degradação probabilística de privacidade em um ataque ACU.
    """
    prior = suc_prior_probabilistic(data, sensitive_attr)
    post = suc_post_probabilistic(data, qid_attrs, sensitive_attr)
    return post / prior if prior > 0 else 0

##### IN_FINANCIAMENTO_ESTUDANTIL

###### Replicando

In [ ]:
qid_attrs = ['CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_FINANCIAMENTO_ESTUDANTIL'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

##### IN_DEFICIENCIA

###### Replicando

In [ ]:
qid_attrs = ['CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")

In [ ]:
qid_attrs = ['NU_DIA_NASCIMENTO', 'NU_MES_NASCIMENTO', 'NU_ANO_NASCIMENTO', 'TP_SEXO', 'TP_COR_RACA', 'CO_MUNICIPIO_NASCIMENTO', 'TP_NACIONALIDADE', 'CO_PAIS_ORIGEM', 'CO_IES', 'CO_CURSO', 'TP_ESCOLA_CONCLUSAO_ENS_MEDIO']
sensitive_attr = 'IN_DEFICIENCIA'
data = microdados

start_time = time.time()

sucesso_priori_deterministic = suc_prior_deterministic(data, sensitive_attr)
sucesso_priori_probabilistic = suc_prior_probabilistic(data, sensitive_attr)
sucesso_post_deterministico = suc_post_deterministic(data, qid_attrs, sensitive_attr)
sucesso_post_probabilistico = suc_post_probabilistic(data, qid_attrs, sensitive_attr)

end_time = time.time()
execution_time = end_time - start_time

print("Sucesso determinístico a priori:", sucesso_priori_deterministic)
print("Sucesso determinístico a posteriori:", sucesso_post_deterministico)
print("Degradação determinística de privacidade:", sucesso_post_deterministico - sucesso_priori_deterministic)

print("Sucesso probabilístico a priori:", sucesso_priori_probabilistic)
print("Sucesso probabilístico a posteriori:", sucesso_post_probabilistico)
print("Degradação probabilística de privacidade:", sucesso_post_probabilistico / sucesso_priori_probabilistic)

print("Tempo de execução:", execution_time, "segundos")